## **Fine Tune LLama using Unsloth**

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
## make a special prompt template for question answering

prompt_format = """Below is a question. Write a response that accurately answers that question.

### Question:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for question, output in zip(question, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt_format.format(question, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("gbharti/finance-alpaca", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/68912 [00:00<?, ? examples/s]

In [6]:
## use supervised fine tuning (SFT) to train the llama3.0 model

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 3,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/68912 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [7]:
## run the trainer

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 68,912 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.772700
2,2.567100
3,2.258500
4,2.127100
5,1.810000
6,2.051600
7,2.166200
8,1.238500
9,1.455500
10,1.599700


In [8]:
torch.cuda.empty_cache()

In [10]:
# test an inference on fine tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_format.format(
        "What are the different types of loans that I can get for scaling my business?", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<|begin_of_text|>Below is a question. Write a response that accurately answers that question.

### Question:
What are the different types of loans that I can get for scaling my business?

### Response:
There are many different types of loans available for businesses looking to scale. The most common are:
1. Business Line of Credit: This type of loan provides a revolving line of credit that can be used for short-term financing needs. It can be used to cover expenses such as payroll, supplies, and marketing costs.
2. Working Capital Loan: This type of loan is used to finance the day-to-day operations of a business. It can be used to cover expenses such as payroll, rent, and inventory.
3. Equipment Financing: This type of loan is used to purchase equipment or machinery for a business. It can be used to cover the cost of purchasing new equipment or replacing existing equipment.
4. Real Estate Loan: This type of loan is used to finance the purchase of a commercial property. It can be used t

In [14]:
## Save the model locally for inference
model.save_pretrained("financial_llama_3.0") # Local saving
tokenizer.save_pretrained("financial-llama_3.0")

('financial-llama_3.0/tokenizer_config.json',
 'financial-llama_3.0/special_tokens_map.json',
 'financial-llama_3.0/tokenizer.json')

In [18]:
## Save the model to huggingface hub

from google.colab import userdata
myToken = userdata.get('MY_TOKEN')

model.push_to_hub("pranav290/financial-Llama_3.0", token = myToken) # Online saving
tokenizer.push_to_hub("pranav290/financial-Llama_3.0", token = myToken) # Online saving

README.md:   0%|          | 0.00/576 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/pranav290/financial-Llama_3.0


In [ ]:
## save the model to huggingface in gguf format

model.push_to_hub_gguf(
    "pranav290/financial_Llama_3.0_gguf",
    tokenizer,
    quantization_method = ["q4_k_m","q8_0","f16"],
    token = myToken
)

In [20]:
!zip /content/outputs.zip /content/outputs

  adding: content/outputs/ (stored 0%)
